In [1]:
from main import *
import datetime as dt

# BIMTRA

In [2]:
# Pega o df do BIMTRA da API
df_bimtra = request_api(endpoint='bimtra', inicial_date = '2022-12-01', final_date ='2023-01-01')

# Transforma os datetimes
df_bimtra[['dt_dep','dt_arr']] = df_bimtra[['dt_dep','dt_arr']]/1000

df_bimtra['dt_dep'] = pd.to_datetime(df_bimtra['dt_dep'], unit ='s')

df_bimtra['dt_arr'] = pd.to_datetime(df_bimtra['dt_arr'], unit ='s')

df_bimtra

,flightid,origem,destino,dt_dep,dt_arr
0,7762d73e399c598a0a4e8d96422cf53f,SBSP,SBBR,2022-12-01 12:00:59,2022-12-01 13:29:19
1,748a7f8c0d0c8121f74029af815f84ad,SBGR,SBBR,2022-12-01 02:29:16,2022-12-01 03:50:55
2,d548a77ec5f3aea78b5f5859bfee9fa3,SBCF,SBBR,2022-12-01 02:45:41,2022-12-01 03:44:48
3,99b91d141f9a459b127382e44b53883a,SBRF,SBBR,2022-12-01 01:47:28,2022-12-01 04:01:25
4,0df635cf7b4cac7e1c9591616474c8de,SBPA,SBBR,2022-12-01 01:45:51,2022-12-01 03:58:15
...,...,...,...,...,...
27189,7ab89601392425d6c01b99c68e2471c8,SBBR,SBRF,2023-01-01 17:08:27,2023-01-01 19:25:32
27190,d73437eefc27178108d74c3d5e217aec,SBBR,SBRF,2023-01-01 11:18:16,2023-01-01 13:28:58
27191,f9a542179b8b864e3a3f45ce0e05ab05,SBSP,SBRF,2023-01-01 10:55:27,2023-01-01 13:35:58
27192,44fa32402beb9a1fcde470d02144f2fa,SBGR,SBCF,2023-01-01 01:17:34,2023-01-01 02:08:40


# CAT-62


In [4]:
# Pega o df do BIMTRA da API
df_cat62 = request_api(endpoint='cat-62', inicial_date ='2022-12-01 00:00:00.000', final_date ='2022-12-02 00:00:00.000')

df_cat62


,flightid,lat,lon,flightlevel,speed,dt_radar
0,5385ee71712e2767734b1a22ba578608,-0.425949,-0.839414,170.0,263.0,1669867742000
1,5385ee71712e2767734b1a22ba578608,-0.416868,-0.827230,145.0,306.0,1669868404000
2,5385ee71712e2767734b1a22ba578608,-0.474318,-0.868783,228.0,216.0,1669865104000
3,5385ee71712e2767734b1a22ba578608,-0.456211,-0.858480,170.0,255.0,1669866064000
4,5385ee71712e2767734b1a22ba578608,-0.467552,-0.864902,171.0,275.0,1669865461000
...,...,...,...,...,...,...
123592,5124aba8991645b29af83e02438dd3cd,-0.409089,-0.811537,28.0,124.0,1669858382000
123593,5124aba8991645b29af83e02438dd3cd,-0.411462,-0.811273,81.0,211.0,1669857606000
123594,5124aba8991645b29af83e02438dd3cd,-0.406259,-0.813279,71.0,241.0,1669857900000
123595,5124aba8991645b29af83e02438dd3cd,-0.409428,-0.812768,40.0,125.0,1669858261000


# ESPERAS

## Duvidas

Esperas? Que diabos é isso?

In [6]:
# Pega o df do ESPERAS DE VOOS da API
df_esperas = request_api(endpoint='esperas', inicial_date = '2022-12-01', final_date ='2023-01-01')

df_esperas[df_esperas['esperas']!=0]

,esperas,hora,aero
8,1,1669881600000,SBBR
10,2,1669888800000,SBBR
20,1,1669924800000,SBBR
134,1,1670335200000,SBBR
160,1,1670428800000,SBBR
...,...,...,...
8439,2,1672585200000,SBSP
8440,1,1672588800000,SBSP
8799,4,1671116400000,SBSV
8914,1,1671530400000,SBSV


# METAF

## Separar em colunas os dados

In [7]:
# Pega o df do ESPERAS DE VOOS da API
df_metaf = request_api(endpoint='metaf', inicial_date = '2022-12-01', final_date ='2023-01-01')

df_metaf

,hora,metaf,aero
0,1669852800000,METAF SBGL 010000Z 07011KT 6000 OVC033 ...,SBGL
1,1669856400000,METAF SBGL 010100Z 08010KT 5000 BR OVC033 ...,SBGL
2,1669860000000,METAF SBGL 010200Z 08010KT 5000 BR OVC016 ...,SBGL
3,1669863600000,METAF SBGL 010300Z 08008KT 4000 BR OVC011 ...,SBGL
4,1669867200000,METAF SBGL 010400Z 08010KT 5000 BR BKN013 ...,SBGL
...,...,...,...
3011,1672599600000,METAF SBSP 011900Z 13007KT 9999 FEW033 ...,SBSP
3012,1672603200000,METAF SBSP 012000Z 12007KT CAVOK 27/16 Q1012=\n,SBSP
3013,1672606800000,METAF SBSP 012100Z 13007KT CAVOK 27/17 Q1013=\n,SBSP
3014,1672610400000,METAF SBSP 012200Z 13005KT CAVOK 25/17 Q1012=\n,SBSP


# METAR

In [8]:
# Pega o df do ESPERAS DE VOOS da API
df_metar = request_api(endpoint='metar', inicial_date = '2022-12-01', final_date ='2023-01-01')

df_metar

,hora,metar,aero
0,1669852800000,METAR SBBR 010000Z 28004KT 9999 FEW045 20/17 Q...,SBBR
1,1669856400000,METAR SBBR 010100Z 29004KT 9999 FEW045 20/17 Q...,SBBR
2,1669860000000,METAR SBBR 010200Z 30004KT 9999 FEW045 20/17 Q...,SBBR
3,1669863600000,METAR SBBR 010300Z 32003KT CAVOK 19/17 Q1013=,SBBR
4,1669867200000,METAR SBBR 010400Z 30004KT CAVOK 18/16 Q1021=,SBBR
...,...,...,...
9953,1672599600000,METAR SBSV 011900Z 12012KT 9999 SCT020 FEW023T...,SBSV
9954,1672603200000,METAR SBSV 012000Z 12010KT 090V200 9999 SCT020...,SBSV
9955,1672606800000,METAR SBSV 012100Z 12009KT 9999 SCT020 SCT080 ...,SBSV
9956,1672610400000,METAR SBSV 012200Z 11007KT 060V170 9999 SCT020...,SBSV


# SATELITE

In [12]:
# Pega o df do ESPERAS DE VOOS da API
df_satelite = request_api(endpoint='satelite', inicial_date = '2022-12-01', final_date ='2023-01-01')

df_satelite

,data,path,tamanho
0,2022-12-01 01:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1794990
1,2022-12-01 02:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1794153
2,2022-12-01 03:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1795192
3,2022-12-01 04:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1788387
4,2022-12-01 05:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1775104
...,...,...,...
745,2023-01-01 19:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1890961
746,2023-01-01 20:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1888224
747,2023-01-01 21:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1868914
748,2023-01-01 22:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1848443


# TC Prev


In [16]:
# Pega o df do ESPERAS DE VOOS da API
df_tcprev = request_api(endpoint='tc-prev', inicial_date = '2022-12-01', final_date ='2023-01-01')

df_tcprev

,hora,troca,aero
0,1669852800000,0,BR
1,1669856400000,0,BR
2,1669860000000,0,BR
3,1669863600000,0,BR
4,1669867200000,0,BR
...,...,...,...
9211,1672599600000,0,SV
9212,1672603200000,0,SV
9213,1672606800000,0,SV
9214,1672610400000,0,SV


# TC Real

In [17]:
# Pega o df do ESPERAS DE VOOS da API
df_tcreal = request_api(endpoint='tc-real', inicial_date = '2022-12-01', final_date ='2023-01-01')

df_tcreal

,hora,nova_cabeceira,antiga_cabeceira,aero
0,1669852980000,17R,29L,SP
1,1669853167000,29L,17R,BR
2,1669879565000,32,14,FL
3,1669880089000,33,15,KP
4,1669900092000,20L,21,RJ
...,...,...,...,...
3025,1672611088000,15,33,CT
3026,1672612272000,15,34,KP
3027,1672612479000,10,17,SV
3028,1672613222000,17,10,SV
